In [1]:
import numpy as np
import math
import random
import cPickle
import time
def unpickle(file):
    fo = open(file, 'rb')
    data = cPickle.load(fo)
    fo.close()
    return data
train_set_1 = unpickle("cifar-10-batches-py/data_batch_1")
train_set_2 = unpickle("cifar-10-batches-py/data_batch_2")
train_set_3 = unpickle("cifar-10-batches-py/data_batch_3")
train_set_4 = unpickle("cifar-10-batches-py/data_batch_4")
train_set_5 = unpickle("cifar-10-batches-py/data_batch_5")
test_set = unpickle("cifar-10-batches-py/test_batch")

In [2]:
X_train_1=train_set_1['data']
Y_train_1=train_set_1['labels']

X_train_2=train_set_2['data']
Y_train_2=train_set_2['labels']

X_train_3=train_set_3['data']
Y_train_3=train_set_3['labels']

X_train_4=train_set_4['data']
Y_train_4=train_set_4['labels']

X_test=test_set['data']
Y_test=test_set['labels']

In [3]:
X_train=np.concatenate((X_train_1,X_train_2,X_train_3,X_train_4),axis=0)
Y_train=np.concatenate((Y_train_1,Y_train_2,Y_train_3,Y_train_4),axis=0)
X_train=X_train.reshape(40000,3,32,32)
X_test=X_test.reshape(10000,3,32,32)

In [4]:
print "length of the X_train ",X_train.shape
print "shape of the X_test ",X_test.shape

length of the X_train  (40000, 3, 32, 32)
shape of the X_test  (10000, 3, 32, 32)


in this version,
i want to change the way how did i div the image.
I will do the things that the artical said.

In [5]:
w = 6
K=1600
whitening=True
nb_patch = 200000
CIFAR_DIM=[32,32,3]

In [6]:
def collect_random_patch(X_train):
    train_patches=[]
    for i in range(nb_patch):
        m=random.randint(0,32-w)
        n=random.randint(0,32-w)
        patch=np.array([])
        image=X_train[i%(len(X_train))]
        for layer in image:
            patch=np.append(patch,layer[m:m+w].T[n:n+w].T.ravel())
        train_patches.append(patch)
    train_patches=np.asarray(train_patches)
    return train_patches
        

In [7]:
train_patches=collect_random_patch(X_train)
print "shape of the train_patches ",train_patches.shape

shape of the train_patches  (200000, 108)


In [8]:
print "normalizing the train patches..."
train_patches=(train_patches-train_patches.mean(1)[:,None])/np.sqrt(train_patches.var(1)+10)[:,None]

normalizing the train patches...


In [9]:
print "whitening the train patches... "
[D,V]=np.linalg.eig(np.cov(train_patches,rowvar=0))

P = V.dot(np.diag(np.sqrt(1/(D + 0.1)))).dot(V.T)
train_patches = train_patches.dot(P)
print "end of whitening"

whitening the train patches... 
end of whitening


In [10]:
from sklearn.cluster import KMeans 
K=800
print "Start creating the dictionary"
print "Start using the sklearn Kmeans...."
result = KMeans(n_clusters=K,max_iter=50).fit(train_patches)
print "K-means has done..."

Start creating the dictionary
Start using the sklearn Kmeans....
K-means has done...


In [11]:
centroids=result.cluster_centers_
print "centroids ",centroids.shape

centroids  (800, 108)


In [20]:
from scipy.spatial import distance
def sliding(img,window=[6,6]):
    out=np.array([])
    for i in range(3):
        row=32
        col=32
        col_extent = col - window[1] + 1
        row_extent = row - window[0] + 1
        start_idx = np.arange(window[0])[:,None]*col + np.arange(window[1])
        offset_idx = np.arange(row_extent)[:,None]*col + np.arange(col_extent)
        if len(out)==0:
            out=np.take (img[i],start_idx.ravel()[:,None] + offset_idx.ravel())
        else:
            out=np.append(out,np.take (img[i],start_idx.ravel()[:,None] + offset_idx.ravel()),axis=0)
    return out


def extract_features_soft(X_train):
    trainXC=[]
    idx=0
    for img in X_train:
        idx+=1
        
        if (idx%1000==0):
            print idx,'/',len(X_train)
        
        patches=sliding(img,[w,w]).T
        patches=(patches-patches.mean(1)[:,None])/(np.sqrt(patches.var(1)+10)[:,None])
        patches=patches.dot(P)
        
        x2=np.power(patches,2).sum(1)
        c2=np.power(centroids,2).sum(1)
        xc=patches.dot(centroids.T)

        dist=np.sqrt(-2*xc+x2[:,None]+c2)
        u=dist.mean(1)
        patches=np.maximum(-dist+u[:,None],0)
        rs=CIFAR_DIM[0]-w+1
        cs=CIFAR_DIM[1]-w+1
        patches=np.reshape(patches,[rs,cs,-1])
        q=[]
        q.append(patches[0:rs/2,0:cs/2].sum(0).sum(0))
        q.append(patches[0:rs/2,cs/2:cs-1].sum(0).sum(0))
        q.append(patches[rs/2:rs-1,0:cs/2].sum(0).sum(0))
        q.append(patches[rs/2:rs-1,cs/2:cs-1].sum(0).sum(0))
        q=np.array(q).ravel()
        trainXC.append(q)
    trainXC=np.array(trainXC)
    trainXC=(trainXC-trainXC.mean(1)[:,None])/(np.sqrt(trainXC.var(1)+0.01)[:,None])
    return trainXC

In [21]:
print "start creating the soft dictionary"
trainXC=extract_features_soft(X_train[0:20000])
print "trainXC compelted"

start creating the soft dictionary
1000 / 20000
2000 / 20000
3000 / 20000
4000 / 20000
5000 / 20000
6000 / 20000
7000 / 20000
8000 / 20000
9000 / 20000
10000 / 20000
11000 / 20000
12000 / 20000
13000 / 20000
14000 / 20000
15000 / 20000
16000 / 20000
17000 / 20000
18000 / 20000
19000 / 20000
20000 / 20000
trainXC compelted


In [22]:
print "start creating the soft dictionary"
testXC=extract_features_soft(X_test)
print "testXC compelted"

start creating the soft dictionary
1000 / 10000
2000 / 10000
3000 / 10000
4000 / 10000
5000 / 10000
6000 / 10000
7000 / 10000
8000 / 10000
9000 / 10000
10000 / 10000
testXC compelted


In [23]:
print "trainXC ",trainXC.shape
print "test XC",testXC.shape

trainXC  (20000, 3200)
test XC (10000, 3200)


After creating the right input, we could choose to our own classify model or could just choose the keras to do this work.

In [24]:
from sklearn.svm import SVC
print "using the triangle dictionary for K-means"
print "using the SVC to do the classification:"
model=SVC()
model=model.fit(trainXC,Y_train[0:20000])

print "model compeleted"


using the triangle dictionary for K-means
using the SVC to do the classification:
model compeleted


In [25]:
print model.score(testXC,Y_test)

0.6049


In [44]:
def extract_features_hard(X_train):
    trainXC=[]
    idx=0
    for img in X_train:
        idx+=1
        
        patches=sliding(img,[w,w]).T

        patches=(patches-patches.mean(1)[:,None])/(np.sqrt(patches.var(1)+10)[:,None])
        patches=patches.dot(P)
        
        pre_index=result.predict(patches)
        dict_patches=np.zeros(shape=(len(patches),K))
        
        for i in range(len(patches)):
            predict_index=pre_index[i]
            dict_patches[i][pre_index]=1
        
        rs=CIFAR_DIM[0]-w+1
        cs=CIFAR_DIM[1]-w+1
        dict_patches=np.reshape(dict_patches,[rs,cs,-1])
        q=[]
        q.append(dict_patches[0:rs/2,0:cs/2].sum(0).sum(0))
        q.append(dict_patches[0:rs/2,cs/2:cs-1].sum(0).sum(0))
        q.append(dict_patches[rs/2:rs-1,0:cs/2].sum(0).sum(0))
        q.append(dict_patches[rs/2:rs-1,cs/2:cs-1].sum(0).sum(0))
        q=np.array(q).ravel()
        trainXC.append(q)
    trainXC=np.array(trainXC)
    trainXC=(trainXC-trainXC.mean(1)[:,None])/(np.sqrt(trainXC.var(1)+0.01)[:,None])
    return trainXC

In [45]:
trainXC_hard=extract_features_hard(X_train[0:20000])
print "trainXC_hard compelted"
print "trainXC_hard ",trainXC_hard.shape
testXC_hard=extract_features_hard(X_test)
print "testXC_hard compelted"
print "testXC_hard ",testXC_hard.shape

trainXC_hard compelted
trainXC_hard  (20000, 3200)
testXC_hard compelted
testXC_hard  (10000, 3200)


In [46]:
from sklearn.svm import SVC
print "using the K-means hard"
print "using the SVC to do the classification: "
model=SVC()
model=model.fit(trainXC_hard,Y_train[0:20000])

print "model compeleted"
print model.score(testXC_hard,Y_test)

using the K-means hard
using the SVC to do the classification: 
model compeleted
0.5905
